In [2]:
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from tensorflow.keras.models import load_model
import tensorflow as tf 

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    print(e)  

In [16]:
def get_embedding(model, face_pixels):
	# scale pixel values
	face_pixels = face_pixels.astype('float32')
	# standardize pixel values across channels (global)
	mean, std = face_pixels.mean(), face_pixels.std()
	face_pixels = (face_pixels - mean) / std
	# transform face into one sample
	samples = expand_dims(face_pixels, axis=0)
	# make prediction to get embedding
	yhat = model.predict(samples)
	return yhat[0]

임베딩을 예측하려면 먼저 FaceNet 모델의 기대치를 충족시키기 위해 이미지 픽셀값을 
적절히 준비해야함. 

In [5]:
# load the face dataset
data = load('5-celebrity-faces-dataset.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)

Loaded:  (93, 160, 160, 3) (93,) (25, 160, 160, 3) (25,)


In [6]:
# load the facenet model
model = load_model('facenet_keras.h5')
print('Loaded Model')

W0405 16:23:07.315459   496 hdf5_format.py:177] No training configuration found in save file: the model was *not* compiled. Compile it manually.


Loaded Model


In [24]:
# convert each face in the train set to an embedding
newTrainX = list()
#i=0
for face_pixels in trainX:
    embedding = get_embedding(model, face_pixels)
    #print('%d'%i,face_pixels)
    newTrainX.append(embedding)
    #i+=1
newTrainX = asarray(newTrainX)
print(newTrainX.shape)

(93, 128)


In [25]:
newTestX = list()
for face_pixels in testX:
	embedding = get_embedding(model, face_pixels)
	newTestX.append(embedding)
newTestX = asarray(newTestX)
print(newTestX.shape)

(25, 128)


In [26]:
savez_compressed('5-celebrity-faces-embeddings.npz', newTrainX, trainy, newTestX, testy)